In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from scipy.stats import t
import matplotlib.pyplot as plt

print("all modules imported successfully")

all modules imported successfully


In [3]:
#load data
def data_loader(file_name: str) -> pd.DataFrame:
    data = pd.read_csv(file_name)
    print(f"Data loaded successfully. Shape: {data.shape}")
    return data

msoft_df = data_loader("microsoft - microsoft.csv")
msoft_df.head()

Data loaded successfully. Shape: (324, 7)


,Date,RPmsoft,RPsandp,Dprod,Dinflation,Dterm,m1
0,1986m5,7.658835,4.376684,0.0960,0.459855,0.26,0
1,1986m6,-13.497473,0.849428,-0.1768,0.273590,0.46,0
2,1986m7,-8.110257,-6.559180,0.3538,-0.549452,-0.40,0
3,1986m8,-0.500000,6.377262,-0.1189,0.182482,0.20,0
4,1986m9,-1.326896,-9.376907,0.0956,0.272271,0.42,0


### Regression function

RPmsof tt = β1+β2RP sandpt+β3Dprodt+β4Dinflationt+β5Dtermt+β6m1t+εt

In [4]:
#2a

#define dependent and independent variables and add constant

X = msoft_df[['RPsandp', 'Dprod', 'Dinflation', 'Dterm', 'm1']]
y = msoft_df['RPmsoft']
X = sm.add_constant(X)

#fit the model
model = sm.OLS(y, X).fit()
beta_hat = model.params[X.columns]
epsilon_hat = model.resid
sigma_hat_squared = (epsilon_hat@ epsilon_hat) / (len(y) - len(beta_hat))
XX_inv = np.linalg.inv(X.T @ X)

deg_freedom = len(y) - len(beta_hat)

print(model.summary())

with open('regression_output.tex', 'w') as f:
    f.write(model.summary().as_latex())


                            OLS Regression Results                            
Dep. Variable:                RPmsoft   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     17.26
Date:                Wed, 22 Oct 2025   Prob (F-statistic):           4.08e-15
Time:                        17:29:07   Log-Likelihood:                -1276.7
No. Observations:                 324   AIC:                             2565.
Df Residuals:                     318   BIC:                             2588.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9291      0.760     -1.223      0.2

In [5]:
# 2b
#Calculate standard errors, t-statistics, and p-values

alpha = 0.01

std_error_beta6 = np.sqrt(sigma_hat_squared * XX_inv[5][5])
t_stat_beta6 = beta_hat.iloc[5] / std_error_beta6
p_value_beta6 = 2 * (1 - t.cdf(np.abs(t_stat_beta6), deg_freedom))

print(f"\nStandard Error of beta_6 (m1): {std_error_beta6}")
print(f"t-statistic of beta_6 (m1): {t_stat_beta6}")
print(f"p-value of beta_6 (m1): {p_value_beta6}")

critical_value_positive = t.ppf(1 - alpha / 2, deg_freedom)
critical_value_negative = t.ppf(alpha / 2, deg_freedom)

print(f"\nCritical t-value (positive): {critical_value_positive}")
print(f"Critical t-value (negative): {critical_value_negative}")

reject = (t_stat_beta6 > critical_value_positive) or (t_stat_beta6 < critical_value_negative)
fail_to_reject = not reject

if reject:
    print(f"At significance level {alpha}, we reject the null hypothesis that beta_6 = 0.")
else:
    print(f"At significance level {alpha}, we fail to reject the null hypothesis that beta_6 = 0.")



Standard Error of beta_6 (m1): 2.8694482721033028
t-statistic of beta_6 (m1): 1.8941512023009712
p-value of beta_6 (m1): 0.05911219106952914

Critical t-value (positive): 2.5913778895151474
Critical t-value (negative): -2.591377889515148
At significance level 0.01, we fail to reject the null hypothesis that beta_6 = 0.


In [6]:
# 2d

# Jarque-Bera test for question 2b

sk = sm.stats.stattools.jarque_bera(epsilon_hat)[2]
kur = sm.stats.stattools.jarque_bera(epsilon_hat)[3]

print(f"\nSkewness of residuals: {sk}")
print(f"Kurtosis of residuals: {kur}")

n = msoft_df.shape[0]
print(n)

JB = (n / 6) * (sk**2 + (1 / 4) * (kur - 3)**2)

print(f"Jarque-Bera statistic: {JB}")

check = JB == sm.stats.stattools.jarque_bera(epsilon_hat)[0]
print(check)

from scipy.stats import chi2

JB_stat = 1809.21
alpha = 0.01
critical_value = chi2.ppf(1 - alpha, df=2)
p_value = 1 - chi2.cdf(JB_stat, df=2)

print(f"\nJarque-Bera Test Results:")
print(f"JB statistic: {JB_stat:.2f}")
print(f"Critical value (α=0.01): {critical_value:.2f}")
print(f"p-value: {p_value}")

if JB_stat > critical_value:
    print(f"\nReject H0: Residuals are NOT normally distributed")
else:
    print(f"\nFail to reject H0: Residuals appear normally distributed")


Skewness of residuals: -2.541093397756985
Kurtosis of residuals: 13.40129942938899
324
Jarque-Bera statistic: 1809.211307998108
True

Jarque-Bera Test Results:
JB statistic: 1809.21
Critical value (α=0.01): 9.21
p-value: 0.0

Reject H0: Residuals are NOT normally distributed
